Import packages

In [1]:
import numpy as np
import copy
import time
import datetime
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

Useful Functions

In [2]:
# image show function
def imshow(x, title=None):
    if torch.is_tensor(x) :
        _x = x.detach().cpu().squeeze()
        if _x.dim() == 2:
            plt.imshow(_x, cmap='gray')
        else:
            plt.imshow(_x)
        plt.title(title)
        plt.show()
    else :
        plt.imshow(x.squeeze())
        plt.title(title)
        plt.show()
        
def create_directory(directory): 
    try: 
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError: print("Error: Failed to create the directory.")
        
def savefig(directory):
    plt.savefig(directory)
    plt.show()
    plt.close()

In [3]:
# Single-Layer Linear and ReLU Networks

class Linear_Net(nn.Module):
    def __init__(self):
        super(Linear_Net, self).__init__()
        self.module = nn.Sequential(
            nn.Linear(d, 1, bias=False)
        )
    def forward(self, x):
        return self.module(x) 
    
    def weight(self):
        return self.module[0].weight.detach().cpu()


class ReLU_Net(nn.Module):
    def __init__(self, output_class=1):
        super(ReLU_Net, self).__init__()
        self.module = nn.Sequential(
            nn.Linear(d, output_class, bias=False),
            nn.ReLU()
        )
    def forward(self, x):
        return self.module(x) 
    
    def weight(self):
        return self.module[0].weight.detach().cpu()

In [4]:
# data loader
def data_load(dataset):
    if dataset == 'MNIST' :
        trn_dataset = torchvision.datasets.MNIST("./Datasets", train=True, download=False,
                                                 transform=torchvision.transforms.ToTensor())
    if dataset == 'CIFAR10' :
        trn_dataset = torchvision.datasets.CIFAR10("./Datasets", train=True, download=False,
                                                 transform=torchvision.transforms.Compose([
                                                     torchvision.transforms.ToTensor(),
                                                     torchvision.transforms.Grayscale(),
                                                 ])
                                                  )

    trn_total_X, trn_total_Y = torch.tensor(trn_dataset.data/255).clone().float(), torch.tensor(trn_dataset.targets).clone().float()
    trn_X = trn_total_X[:n].reshape(n, -1) # n x d matrix
    trn_Y = trn_total_Y[:n].reshape(n, 1) # n x 1 vector
    return trn_X, trn_Y